In [28]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
import cv2

import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rcParams['figure.dpi'] = 300

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import gen_theta_set

from bo_methods_lib.CS2_bo_plotters import plot_obj_abs_min, value_plotter, plot_obj, plot_Theta, plot_Theta_min, path_name, plot_org_train, save_fig

from bo_methods_lib.CS2_bo_functions_multi_dim import set_ep
from pathlib import Path

In [3]:
# DateTime = "2022/10/21/13-05" #Date and Time - Dispayed in job file
DateTime = "2022/10/21/13-18" #Date and Time - Dispayed in job file
# DateTime = None

exp_d = 1
n = 5

param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
indecies = [list(param_dict.keys())[list(param_dict.values()).index('\\theta_1')],
            list(param_dict.keys())[list(param_dict.values()).index('\\theta_2')]]

Theta_True = np.array([1,-1])

bounds = np.array([[-2, -2],
                   [ 2,  2]])


# exp_d = 2
# n = 15 #Number of experimental data points to use

# param_dict = {0 : 'a_1', 1 : 'a_2', 2 : 'a_3', 3 : 'a_4',
#               4 : 'b_1', 5 : 'b_2', 6 : 'b_3', 7 : 'b_4'}
# indecies = [list(param_dict.keys())[list(param_dict.values()).index('a_1')],
#             list(param_dict.keys())[list(param_dict.values()).index('a_2')]]

# Constants = np.array([[-200,-100,-170,15],
#                       [-1,-1,-6.5,0.7],
#                       [0,0,11,0.6],
#                       [-10,-10,-6.5,0.7],
#                       [1,0,-0.5,-1],
#                       [0,0.5,1.5,1]])

# Theta_True = Constants[1:3].flatten()

# bounds = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
#                    [ 2,  2,   0,  2,  2,  2, 15,  2]])

param_names_list = [param_dict[0], param_dict[1]] #Fill in depending on problem
mesh_combo = str(param_names_list[0]) + "-" + str(param_names_list[1])

set_lengthscale = None
save_CSV = False

#Define GP Testing space
p=20
d = len(Theta_True)

# Need to redefine to save theta_set_org 
# theta_set_org = gen_theta_set(LHS = True, n_points = p, dimensions = d, bounds = bounds)
# Theta1_lin = np.linspace(np.min(theta_set_org[:,indecies[0]]),np.max(theta_set_org[:,indecies[0]]), p)
# Theta2_lin = np.linspace(np.min(theta_set_org[:,indecies[1]]),np.max(theta_set_org[:,indecies[1]]), p)
# theta_mesh = np.array(np.meshgrid(Theta1_lin, Theta2_lin)) 

theta_mesh = gen_theta_set(LHS = False, n_points = p, dimensions = d, bounds = bounds)
theta_mesh = theta_mesh.T.reshape((2, p, p))
# print(theta_mesh.shape)

# tot_iter = bo_iters= 3 #Total Number of iters - Dispayed in job file
# tot_runs = 2 #Total Number of runs - Dispayed in job file

tot_iter = bo_iters= 100 #Total Number of iters - Dispayed in job file
tot_runs = 15 #Total Number of runs - Dispayed in job file

is_figure = True
save_figure = False

In [36]:
def make_5_fig_plots(path_list, sep_fact, title_save, Img_names, save_figure = False, close = False, row_num = 3):
    # code for displaying multiple images in one figure
#     save_figure = True
    #import libraries
    # Img_names = ["NLR", "Method 1A", "Method 1B", "Method 2A", "Method 2B", "Method 2C"]
    # setting values to rows and column variables
    rows = row_num
    columns = 2

    # reading images
    Images = []
    for i in range(len(path_list)):
        Img = cv2.imread(path_list[i])
        Img = cv2.cvtColor(Img, cv2.COLOR_BGR2RGB)
        Images.append(Img)

    # create figure
    fig = plt.figure(figsize=(6, 8))

    for i in range(len(Images)):
        # Adds a subplot at the 1st position
        fig.add_subplot(rows, columns, i+1)

        # showing image
        plt.imshow(Images[i])
        plt.axis('off')
        plt.title(Img_names[i], fontsize = 16)

#     if save_figure == True:
#         if sep_fact is not None:
#             save_to_path = "../6_fig_plots/CS1/"+"sep_fact_"+str(np.round(sep_fact,1))+"/"+str(title_save)+"_maps"
#         else:
#             save_to_path = "../6_fig_plots/CS1/"+str(title_save)+"_maps"
#         save_fig(save_to_path, ext='png', close=close, verbose=False)
        
    if save_figure == True:
        if sep_fact is not None:
            save_to_path = "../6_fig_plots/CS1/"+str(title_save)+"_walkthrough/Iter_"+str(sep_fact)
        else:
            #Note: "Sep_fact" will just be EI iters instead
            save_to_path = "../6_fig_plots/CS1/"+str(title_save)+"_maps"
#         print(save_to_path)
        save_fig(save_to_path, ext='png', close=close, verbose=False)
    return


In [37]:
#Define Factors for BO images and paths for all 4 images (EI Heat Maps)
fxn = "value_plotter"
title_save = "EI"
title = 'E(I(\\theta))'
# title = 'log(E(I(\\theta)))'
# title_save = "ln(SSE)"
# title = 'log(e(\\theta))'

emulator =    [False, False,   True, True,     True]
obj =         ["obj", "LN_obj","obj", "LN_obj", "obj" ]
sparse_grid = [False, False,   False, False,    True]
sep_fact =    [0.1,    1.0,     0.1, 0.2,      0.7]
Img_names = ["Method 1A", "Method 1B", "Method 2A", "Method 2B", "Method 2C"]

mesh_combo_list = [mesh_combo, None, mesh_combo, None, mesh_combo]
t0 = 20
t = []
ep = []
for i in range(len(sep_fact)):
    ep.append(set_ep(emulator[i], obj[i], sparse_grid[i]))
    if emulator[i] == True:
        t_new = n*t0
    else:
        t_new = t0
    t.append(t_new)

run = np.array([3, 8, 11, 11 ,12])
Bo_iter = np.array([45, 100, 7, 6, 6])
Bo_iter_array = np.array([[45, 100, 7, 6, 6],
                          [1, 1, 1, 1, 1],
                          [25, 25, 25, 25, 2],
                          [50, 50, 50, 50, 4],
                          [75, 75, 75, 75, 6],
                          [100, 100, 100, 100, 8]])

run -= 1 # run should be number you want -1
Bo_iter -= 1 #Iter you want -1
Bo_iter_array -= 1 #Iter you want -1

Iter_list_name = ["best", "first", "1-quart", "half", "3-quart", "last" ]
for j in range(len(Bo_iter_array)):
    path_list = []
    Bo_iter_list = Bo_iter_array[j]
    for i in range(len(Bo_iter_list)):
        path_title = "../"+ path_name(emulator[i], ep[i], sparse_grid[i], fxn, set_lengthscale, t[i], obj[i], mesh_combo_list[i], 
                               Bo_iter_list[i], title_save, run[i], tot_iter=tot_iter, tot_runs=tot_runs, DateTime=DateTime, 
                               sep_fact = sep_fact[i], is_figure = is_figure)
        full_path = Path(path_title + ".png")
        if full_path.is_file() == False:
            path_title = path_title.replace("/\\theta_1-\\theta_2", "")
#         print(path_title)
        path_list.append(path_title + ".png")
#     print(len(path_list))
#     print(path_list)
    save_figure = False
    save_figure = True
    make_5_fig_plots(path_list, Iter_list_name[j], title_save, Img_names, save_figure = save_figure, close = True, row_num  = 3)